# Codificación con base en la media

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
ruta='/content/drive/MyDrive/Ingeniería y Ciencia de Datos/Segundo Semestre/Ingeniería de Características/Colab Notebooks/'

data=pd.read_csv(ruta+'temps.txt',sep='\s+',header=None) # Fuente: http://astro.iam.udg.mx/Estacion/NOAAPRMO.TXT
                                                         #         http://astro.iam.udg.mx/Estacion/NOAAMO.TXT
data.columns=['Temperatura máxima','Temperatura mínima','HDD','CDD','Rain']
d=['Sábado','Domingo','Lunes','Martes','Miércoles','Jueves','Viernes']
dias=[d[i%7] for i in range(len(data))]
catego=['Soleado','Mayormente soleado','Parcialmente nublado','Mayormente nublado','Nublado','Lluvioso']
prono=[catego[-1] if data['Rain'][i]>0 else catego[int(round(1.2*(data['HDD'][i]-data['CDD'][i])+2))] for i in range(len(data))]
df=pd.DataFrame(columns=['Días','Pronóstico','Temperatura máxima','Temperatura mínima'],index=range(len(data)))
df.iloc[:,2:4]=data.iloc[:,0:2]
df['Días']=dias
df['Pronóstico']=prono
df.head()

,Días,Pronóstico,Temperatura máxima,Temperatura mínima
0,Sábado,Parcialmente nublado,28.2,11.2
1,Domingo,Parcialmente nublado,29.1,10.7
2,Lunes,Mayormente nublado,26.1,10.4
3,Martes,Mayormente soleado,29.4,11.1
4,Miércoles,Mayormente soleado,29.8,11.1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title Usando el promedio
# Codificamos la variable 'Pronóstico' usando la variable 'Temperatura mínima'
df['Pronóstico'].unique()

array(['Parcialmente nublado', 'Mayormente nublado', 'Mayormente soleado',
       'Soleado', 'Lluvioso', 'Nublado'], dtype=object)

In [ ]:
media_pronostico=df.groupby('Pronóstico')['Temperatura mínima'].mean()
media_pronostico

Pronóstico
Lluvioso                12.850000
Mayormente nublado      10.683333
Mayormente soleado      11.435714
Nublado                  9.500000
Parcialmente nublado    11.261111
Soleado                 11.800000
Name: Temperatura mínima, dtype: float64

In [ ]:
df['media_mín_Pronóstico']=df['Pronóstico'].map(media_pronostico)
df.head()

,Días,Pronóstico,Temperatura máxima,Temperatura mínima,media_mín_Pronóstico
0,Sábado,Parcialmente nublado,28.2,11.2,11.261111
1,Domingo,Parcialmente nublado,29.1,10.7,11.261111
2,Lunes,Mayormente nublado,26.1,10.4,10.683333
3,Martes,Mayormente soleado,29.4,11.1,11.435714
4,Miércoles,Mayormente soleado,29.8,11.1,11.435714


In [ ]:
#@title Usando una regresión lineal de la codificación de Efectos
#
# Codificación de Efectos
E=pd.get_dummies(df,columns=['Pronóstico'],prefix='',prefix_sep='',drop_first=True) # Hasta aquí teneís la codificación Ficticia
CR,=set(df['Pronóstico'].unique())-set(E.columns[4:])
CR

'Lluvioso'

In [ ]:
idx_CR,=np.where(df['Pronóstico']==CR)
E.iloc[idx_CR,4:]=-1

In [ ]:
linE=LinearRegression()
linE.fit(E.iloc[:,4:],E['Temperatura mínima'])

LinearRegression()

In [ ]:
# Usamos los coeficientes de la regresión lineal asociados a cada categoría :D
E.columns[4:],linE.coef_

(Index(['Mayormente nublado', 'Mayormente soleado', 'Nublado',
        'Parcialmente nublado', 'Soleado'],
       dtype='object'),
 array([-0.57169312,  0.18068783, -1.75502646,  0.00608466,  0.54497354]))

In [ ]:
regLin_E={categoria:coeficiente for categoria,coeficiente in zip(E.columns[4:],linE.coef_)}
regLin_E

{'Mayormente nublado': -0.5716931216931218,
 'Mayormente soleado': 0.18068783068783112,
 'Nublado': -1.7550264550264556,
 'Parcialmente nublado': 0.006084656084656401,
 'Soleado': 0.5449735449735458}

In [ ]:
df['regLinEfectos_Pronóstico']=df['Pronóstico'].map(regLin_E)
df

,Días,Pronóstico,Temperatura máxima,Temperatura mínima,regLinEfectos_Pronóstico
0,Sábado,Parcialmente nublado,28.2,11.2,0.006085
1,Domingo,Parcialmente nublado,29.1,10.7,0.006085
2,Lunes,Mayormente nublado,26.1,10.4,-0.571693
3,Martes,Mayormente soleado,29.4,11.1,0.180688
4,Miércoles,Mayormente soleado,29.8,11.1,0.180688
5,Jueves,Mayormente soleado,30.2,10.2,0.180688
6,Viernes,Mayormente soleado,29.3,10.3,0.180688
7,Sábado,Mayormente soleado,28.6,10.9,0.180688
8,Domingo,Parcialmente nublado,29.0,9.8,0.006085
9,Lunes,Mayormente soleado,27.2,11.2,0.180688


In [ ]:
regLin_E[CR]=linE.intercept_
df['regLinEfectos_Pronóstico']=df['Pronóstico'].map(regLin_E)
df

,Días,Pronóstico,Temperatura máxima,Temperatura mínima,regLinEfectos_Pronóstico
0,Sábado,Parcialmente nublado,28.2,11.2,0.006085
1,Domingo,Parcialmente nublado,29.1,10.7,0.006085
2,Lunes,Mayormente nublado,26.1,10.4,-0.571693
3,Martes,Mayormente soleado,29.4,11.1,0.180688
4,Miércoles,Mayormente soleado,29.8,11.1,0.180688
5,Jueves,Mayormente soleado,30.2,10.2,0.180688
6,Viernes,Mayormente soleado,29.3,10.3,0.180688
7,Sábado,Mayormente soleado,28.6,10.9,0.180688
8,Domingo,Parcialmente nublado,29.0,9.8,0.006085
9,Lunes,Mayormente soleado,27.2,11.2,0.180688
